In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import feather

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import datetime, os
import math
import gc
import sys

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import regularizers
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import mixed_precision
from tensorflow.keras.callbacks import Callback

from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity

from multiprocessing import Pool
import tensorflow as tf
from scipy.stats import ttest_ind
from scipy.ndimage.filters import gaussian_filter1d


In [ ]:
RNG_SEED = 517

# Download data

In [ ]:
def prepare_dataset(year):
    data = pd.read_stata("data/ed" + str(year) + "-stata.dta", convert_categoricals=False)
    return data

years = [(2009,), (2010,), (2011,), (2012,), (2013,), (2014,), (2015,), (2016,), (2017,), (2018,), (2019,), (2020,)]
 
final_data = [None] * len(years)

with Pool(processes=len(years)) as pool:
    df_list = [pool.apply_async(prepare_dataset, p) for p in years]
    for i in range(0, len(years)):
        final_data[i] = df_list[i].get()

    pool.close()

In [ ]:
# Fill missing meds
for i in range(1, 30):
  name = 'MED' + str(i)
  for df in final_data:
    if name not in df:
      df[name] = -9

In [ ]:
data = pd.concat(final_data)
data = data.reset_index()

In [ ]:
meds_types = {}
for i in range(1, 30):
  name = 'MED' + str(i)
  meds_types[name] = 'int32'

data = data.astype(meds_types)

# Descriptive analysis

## Meta data

In [ ]:
icd10_orpha = {
    '5819-': 34145,
    'N04-': 69061,
    'N05-': 28588,
    '5839-': 567544,
    'N059': 567544, # Not exactly, the etiology isn't known yet
    'N119': 93111, # Not exactly, the etiology isn't known yet
    'C64-': 319276,
    '1899-': 319276,
    'C649': 319276,
    'E72-': 213,
    'E834': 2196
}

In [ ]:
data["RISK"] = 0
data["FINALDIAG"] = "0"

for code in icd10_orpha:
  bool_arr = data["HDDIAG1"].str.match(code) | data["HDDIAG"].str.match(code) | data["HDDIAG2"].str.match(code) | data["HDDIAG3"].str.match(code) | data["HDDIAG4"].str.match(code) | data["HDDIAG5"].str.match(code)
  data.loc[bool_arr, "RISK"] = 1
  data.loc[bool_arr, "FINALDIAG"] = code

In [ ]:
# Remove metadata about the event
data.drop(columns=['VMONTH', 'VDAYR', 'ARRTIME', 'WAITTIME', 'AGEDAYS', 'AGER', 'VYEAR', 'YEAR'], errors='ignore', inplace=True)
# Remove metadata regarding finances
data.drop(columns=['NOPAY', 'PAYPRIV', 'PAYMCARE', 'PAYMCAID', 'PAYWKCMP', 'PAYSELF', 'PAYNOCHG', 'PAYOTH', 'PAYDK', 'PAYTYPER'], errors='ignore', inplace=True)
# Remove preprocessed meta that the model will figure out
data.drop(columns=['NOCHRON', 'TOTCHRON', 'DIAGSCRN', 'TOTDIAG', 'PROC', 'TOTPROC'], errors='ignore', inplace=True)
# Remove random data
data.drop(columns=['index', 'BLANK1', 'BLANK2', 'BLANK3', 'BLANK4', 'SURGDAY', 'RFV13D', 'RFV23D', 'RFV33D', 'RFV43D', 'RFV53D'], errors='ignore', inplace=True)
# Remove exit data
data.drop(columns=['NUMGIV', 'HDSTAT', 'ADISP'], errors='ignore', inplace=True)
# Drop exit medication
data = data.loc[:,~data.columns.str.startswith('GPMED')]
data = data.loc[:,~data.columns.str.startswith('DRUGID')]
data = data.loc[:,~data.columns.str.startswith('PRESCR')]
data = data.loc[:,~data.columns.str.startswith('CONTSUB')]
data = data.loc[:,~data.columns.str.startswith('COMSTAT')]
data = data.loc[:,~data.columns.str.startswith('RX')]
# Drop hospital coding
data.drop(columns=['HOSPCODE', 'PATCODE', 'EMRED', 'HHSMUE', 'EHRINSE', 'EDPRIM', 'EDINFO', 'OBSCLIN', 'OBSSEP', 'OBSPHYSED', 'OBSHOSP', 
'OBSPHYSOT', 'OBSPHYSUN', 'BOARD', 'BOARDHOS', 'AMBDIV', 'TOTHRDIVR', 'REGDIV', 'ADMDIV', 'INCSHX', 'INCPHYS', 'EXPSPACE', 'BEDREG', 'KIOSELCHK', 
'CATRIAGE', 'IMBED', 'ADVTRIAGE', 'PHYSPRACTRIA', 'FASTTRAK', 'EDPTOR', 'DASHBORD', 'RFID', 'WIRELESS', 'ZONENURS', 'POOLNURS', 'SURGDAY',
'BEDCZAR', 'BEDDATA', 'HLIST', 'HLISTED', 'EMEDRES', 'REGION', 'MSA', 'SETTYPE', 'CSTRATM', 'CPSUM', 'PATWT', 'EDWT', 'ADVTRIAG', 'MED'], errors='ignore', inplace=True)

# Drop diags
data = data.loc[:,~data.columns.str.startswith('HDDIAG')]
data = data.loc[:,~data.columns.str.startswith('DIAG')]
data = data.loc[:,~data.columns.str.startswith('PRDIAG')]

# Drop bad columns
data.replace("", np.NaN, inplace=True)
data.dropna(how='any', axis='columns', inplace=True)

In [ ]:
features = ['TEMPF', 'PULSE', 'RESPR', 'BPSYS', 'BPDIAS', 'POPCT']
vcount = 0
for f in features:
  vcount = vcount + data[f].value_counts()[-9]
  data[f].replace(-9, np.NaN, inplace=True)
  data[f].replace(998, np.NaN, inplace=True)

data = data.interpolate(method='linear', limit_direction ='forward')
data = data.reset_index()

excluded_indices = data.loc[data['RISK'] == 1]

## Prepare data for processing

### Medication

In [ ]:
med_list = []
for i in range(1, 30):
    med_list.append('MED' + str(i))

meds = data[med_list].values.tolist()

medication_binarizer = MultiLabelBinarizer(sparse_output=True)
meds = pd.DataFrame.sparse.from_spmatrix(medication_binarizer.fit_transform(meds), columns=medication_binarizer.classes_,index=data.index).drop(columns=[-9]).add_prefix('med_').astype(np.float16)
meds = meds.sparse.to_dense()

del medication_binarizer

### Reason for visit

In [ ]:
rfv_list = []
for i in range(1, 4):
    rfv_list.append('RFV' + str(i))

rfvs = data[rfv_list].values.tolist()

rfv_binarizer = MultiLabelBinarizer(sparse_output=True)
rfvs = pd.DataFrame.sparse.from_spmatrix(rfv_binarizer.fit_transform(rfvs),columns=rfv_binarizer.classes_,index=data.index).drop(columns=[-9]).add_prefix('rfv_').astype(np.float16)
rfvs = rfvs.sparse.to_dense()

del rfv_binarizer

### Causes



In [ ]:
cause_list = []
for i in range(1, 4):
    cause_list.append('CAUSE' + str(i))

causes = data[cause_list].values.tolist()

cause_binarizer = MultiLabelBinarizer(sparse_output=True)
causes = pd.DataFrame.sparse.from_spmatrix(cause_binarizer.fit_transform(causes),columns=cause_binarizer.classes_,index=data.index).drop(columns=[-9, -8], errors='ignore').add_prefix('causes_').astype(np.float16)
causes = causes.sparse.to_dense()

del cause_binarizer

### Features


In [ ]:
scalers = {}  

individual_ft = pd.DataFrame(index=data.index)

individual_features = ['ETHUN', 'RESIDNCE', 'RACEUN', 'SEEN72', 'EPISODE', 'INJURY', 'PAINSCALE']

for f in individual_features:
    scalers[f] = LabelBinarizer()
    temp_cp = pd.DataFrame(scalers[f].fit_transform(data[f]), columns=scalers[f].classes_, index=data.index).drop(columns=[-9, -8], errors='ignore').add_prefix(f + '_').astype(np.float16)
    individual_ft = individual_ft.join(temp_cp)

scalers['SEX'] = LabelBinarizer()
individual_ft['SEX'] = scalers['SEX'].fit_transform(data['SEX'])

In [ ]:
numeric_ft = pd.DataFrame(index=data.index)

data[features] = data[features].replace(to_replace=-9.0, value=np.NaN)

for f in features:
    scalers[f] = StandardScaler()
    t = np.asarray(data[f])
    t = t.reshape(-1,1)
    temp_cp = scalers[f].fit_transform(t)
    temp_cp = pd.DataFrame(temp_cp, index=data.index, columns=[f])
    
    numeric_ft[f + '_-10'] = temp_cp[f] < -10
    numeric_ft[f + '_-5'] = (temp_cp[f] < -5) & (temp_cp[f] >= -10)
    numeric_ft[f + '_-3'] = (temp_cp[f] < -3) & (temp_cp[f] >= -5)
    numeric_ft[f + '_-1'] = (temp_cp[f] < -1) & (temp_cp[f] >= -3)
    numeric_ft[f + '_0'] = (temp_cp[f] < 1) & (temp_cp[f] >= -1)
    numeric_ft[f + '_1'] = (temp_cp[f] > 1) & (temp_cp[f] <= 3)
    numeric_ft[f + '_3'] = (temp_cp[f] > 3) & (temp_cp[f] <= 5)
    numeric_ft[f + '_5'] = (temp_cp[f] > 5) & (temp_cp[f] <= 10)
    numeric_ft[f + '_10'] = temp_cp[f] > 10

numeric_ft.replace(to_replace=False, value=0, inplace=True)
numeric_ft.replace(to_replace=True, value=1, inplace=True)

### Labs

In [ ]:
lab_features = ['CEBVD', 'CHF', 'EDHIV', 'CBC', 'BUNCREAT', 'CARDENZ', 'ELECTROL', 'GLUCOSE', 'LFT', 'ABG', 'PTTINR', 'BLOODCX', 'BAC', 'OTHERBLD', 'CARDMON', 'EKG', 'HIVTEST', 'FLUTEST', 'PREGTEST', 'TOXSCREN', 'URINE', 'WOUNDCX', 'OTHRTEST', 'ANYIMAGE', 'XRAY', 'CATSCAN', 'CTHEAD', 'CTUNK', 'MRI', 'ULTRASND', 'OTHIMAGE', 'IVFLUIDS', 'SUTURE', 'INCDRAIN', 'NEBUTHER', 'BLADCATH', 'PELVIC', 'CENTLINE', 'CPR', 'ENDOINT', 'OTHPROC']

In [ ]:
dist_features = ['OBSHOS', 'ADMITHOS', 'OBSDIS', 'TRANOTH', 'TRANPSYC']

### Autoencode

In [ ]:
test_meds_positive = meds.iloc[excluded_indices.index]
meds.drop(index=excluded_indices.index, inplace=True)

test_meds_negative = meds.sample(frac=0.2, random_state=RNG_SEED)
meds.drop(index=test_meds_negative.index, inplace=True)

In [ ]:
test_rfvs_positive = rfvs.iloc[excluded_indices.index]
rfvs.drop(index=excluded_indices.index, inplace=True)

test_rfvs_negative = rfvs.sample(frac=0.2, random_state=RNG_SEED)
rfvs.drop(index=test_rfvs_negative.index, inplace=True)

In [ ]:
test_causes_positive = causes.iloc[excluded_indices.index]
causes.drop(index=excluded_indices.index, inplace=True)

test_causes_negative = causes.sample(frac=0.2, random_state=RNG_SEED)
causes.drop(index=test_causes_negative.index, inplace=True)

In [ ]:
lab_features = data.loc[:,lab_features]

test_lab_features_positive = lab_features.iloc[excluded_indices.index]
lab_features.drop(index=excluded_indices.index, inplace=True)

test_lab_features_negative = lab_features.sample(frac=0.2, random_state=RNG_SEED)
lab_features.drop(index=test_lab_features_negative.index, inplace=True)

In [ ]:
dist_features = data.loc[:,dist_features]

test_dist_features_positive = dist_features.iloc[excluded_indices.index]
dist_features.drop(index=excluded_indices.index, inplace=True)

test_dist_features_negative = dist_features.sample(frac=0.2, random_state=RNG_SEED)
dist_features.drop(index=test_dist_features_negative.index, inplace=True)

In [ ]:
test_numeric_positive = numeric_ft.iloc[excluded_indices.index]
numeric_ft.drop(index=excluded_indices.index, inplace=True)

test_numeric_negative = numeric_ft.sample(frac=0.2, random_state=RNG_SEED)
numeric_ft.drop(index=test_numeric_negative.index, inplace=True)

In [ ]:
training_df = meds.join(rfvs).join(causes).join(lab_features).join(numeric_ft).join(dist_features).astype(np.int8)

In [ ]:
test_neg_df = test_meds_negative.join(test_rfvs_negative).join(test_causes_negative).join(test_lab_features_negative).join(test_numeric_negative).join(test_dist_features_negative)
test_pos_df = test_meds_positive.join(test_rfvs_positive).join(test_causes_positive).join(test_lab_features_positive).join(test_numeric_positive).join(test_dist_features_positive)

test_df = pd.concat([test_neg_df, test_pos_df])

In [ ]:
def get_f1(y_true, y_pred): #taken from old keras source code
    y_true = K.cast(y_true, 'float32')

    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return K.mean(f1_val)

def get_specificity(true, pred):
    true = K.cast(true, 'float32')

    ground_positives = K.sum(true, axis=0) + K.epsilon()       # = TP + FN
    pred_positives = K.sum(pred, axis=0) + K.epsilon()         # = TP + FP
    true_positives = K.sum(true * pred, axis=0) + K.epsilon()  # = TP
    true_negatives = K.sum((1-true) * (1-pred), axis=0) + K.epsilon()  # = TN
    false_positives = K.sum((1-true) * pred, axis=0) + K.epsilon()  # = FP
    
    precision = true_positives / pred_positives 
    recall = true_positives / ground_positives

    specificity = true_negatives / (true_negatives + false_positives + K.epsilon())

    f1 = 2 * (precision * recall) / (precision + recall + K.epsilon())

    weighted_f1 = f1 * ground_positives / K.sum(ground_positives) 
    weighted_f1 = K.sum(weighted_f1)

    return specificity

def f1_weighted(true, pred): #shapes (batch, 4)
    true = K.cast(true, 'float32')

    ground_positives = K.sum(true, axis=0) + K.epsilon()       # = TP + FN
    pred_positives = K.sum(pred, axis=0) + K.epsilon()         # = TP + FP
    true_positives = K.sum(true * pred, axis=0) + K.epsilon()  # = TP
        #all with shape (4,)
    
    precision = true_positives / pred_positives 
    recall = true_positives / ground_positives
        #both = 1 if ground_positives == 0 or pred_positives == 0
        #shape (4,)

    f1 = 2 * (precision * recall) / (precision + recall + K.epsilon())
        #still with shape (4,)

    weighted_f1 = f1 * ground_positives / K.sum(ground_positives) 
    weighted_f1 = K.sum(weighted_f1)

    return 1 - weighted_f1 #for metrics, return only 'weighted_f1'

In [ ]:
def create_autoencoder(input_count, layers):
  input = keras.Input(shape=(input_count,))
  features = input

  i = 0
  for h in layers:
    features = keras.layers.Dense(h, activation='tanh', kernel_regularizer=regularizers.L1(0.0001))(features)
    if (len(layers) - 1) / 2 == i:
      encoder_dim = h
      encoder = features
    i = i + 1
    features = keras.layers.BatchNormalization()(features)
    #features = keras.layers.Dropout(0.1)(features)
  decoded = keras.layers.Dense(input_count, activation='sigmoid', bias_initializer=keras.initializers.Constant(0.01))(features)
  
  full_model = keras.Model(input, decoded)
  encoder = keras.Model(input, encoder)

  encoded_input = keras.Input(shape=(encoder_dim,))

  return full_model, encoder

In [ ]:
def exp_decay(epoch):
   initial_lrate = 0.00001
   k = 0.04
   lrate = initial_lrate * math.exp(-k*epoch)
   return lrate

In [ ]:
class ClearMemory(Callback):
    def on_epoch_end(self, epoch, logs=None):
        gc.collect()
        K.clear_session()

In [ ]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
hsitoryfile = '/content/drive/MyDrive/Medicine/MRC/Models/history_' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + '.csv'

history_callback = keras.callbacks.CSVLogger(hsitoryfile)
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

early_callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

lrate = keras.callbacks.LearningRateScheduler(exp_decay)

full_ae, full_encoder = create_autoencoder(training_df.shape[-1], [8000, 3000, 1500, 700, 300, 90, 300, 700, 1500, 3000, 8000])
full_ae.compile(optimizer=keras.optimizers.Adam(learning_rate=0.00001), run_eagerly=None,
                loss=keras.losses.binary_crossentropy)

checkpoint_callback = ModelCheckpoint('/content/drive/MyDrive/Medicine/MRC/Models/full-11y-90.best.keras', monitor='loss', save_best_only=True, mode='min')

full_ae.fit(x=training_df, y=training_df, batch_size=64, epochs=1000, use_multiprocessing=True, workers=6, validation_split=0.12, callbacks=[ClearMemory(), lrate, checkpoint_callback, early_callback, history_callback], verbose=1)

In [ ]:
def plot_loss(history, label, n):

  loss = gaussian_filter1d(history['loss'], sigma=2)

  # Use a log scale to show the wide range of values.
  plt.semilogy(history.epoch, loss,
               color=colors[n], label='Train '+label)
  
  if 'val_loss' in history:
    val_loss = gaussian_filter1d(history['val_loss'], sigma=2)
    plt.semilogy(history.epoch, val_loss,
            color=colors[n], label='Validation '+label,
            linestyle="--")
  plt.xlabel('Epoch')
  plt.ylabel('Loss')

  plt.legend()

In [ ]:
test_predictions_baseline = full_ae.predict(test_pos_df, batch_size=64)
test_neg_predictions_baseline = full_ae.predict(test_neg_df, batch_size=64)

In [ ]:
bcount = len(test_neg_predictions_baseline)
losses = np.zeros(bcount)

bce = tf.keras.losses.BinaryCrossentropy(from_logits=False,
    reduction=tf.keras.losses.Reduction.NONE)

for index in range(0, bcount):
  losses[index] = bce(test_neg_predictions_baseline[index], test_neg_df.iloc[index])

p_losses = bce(test_predictions_baseline, test_pos_df).numpy()

In [ ]:
positive_analysis = data.iloc[test_pos_df.index].copy()

In [ ]:
p_losses_df = pd.DataFrame(index=test_pos_df.index, data=p_losses)
positive_analysis["SCORE"] = p_losses_df[0]

In [ ]:
def generate_metrics(cutoff):
  fp = (losses > cutoff).sum()
  tn = len(test_neg_predictions_baseline) - fp

  tp = (p_losses > cutoff).sum()
  fn = len(test_predictions_baseline) - tp

  prevalence = (tp + fn) / (tp + fn + tn + fp)
  sensitivity = tp / (tp + fn)
  fnr = fn / (tp + fn)
  tnr = tn / (fp + tn)
  fallout = fp / (fp + tn)
  ppv = tp / (tp + fp)
  npv = tn / (fn + tn)

  lr_p = sensitivity / fallout
  lr_n = fnr / tnr
  dor = lr_p / lr_n
  f_score = 2 * (ppv * sensitivity) / (ppv + sensitivity)

  return prevalence, tnr, sensitivity, fallout, ppv, npv, lr_p, lr_n, dor, f_score

In [ ]:
cutoff = 0.52

prevalence, tnr, sensitivity, fallout, ppv, npv, lr_p, lr_n, dor, f_score = generate_metrics(cutoff)

print("cutoff: ", cutoff)
print("Prevalence ", prevalence)
print("Specificity ", tnr)
print("True positive rate ", sensitivity)
print("False positive rate ", fallout)
print("PPV ", ppv)
print("NPV ", npv)
print("LR+ ", lr_p)
print("LR- ", lr_n)
print("DOR ", dor)
print("F-score ", f_score)

In [ ]:
plt.rcParams['figure.figsize'] = [12, 12]
plt.rcParams['figure.dpi'] = 100 # 200 e.g. is really fine, but slower

sns.set(font_scale = 2)

In [ ]:
sns.histplot(data=losses, label="Negative cases", color='darkorange', kde=True, stat='density', cbar_kws={'edgecolor':'black'}, line_kws={'linewidth': 4})
sns.histplot(data=p_losses, label="Positive cases", color='darkblue', kde=True, stat='density', cbar_kws={'edgecolor':'black'}, line_kws={'linewidth': 4}, bins=25)

plt.axvline(cutoff, color='black', label="Test cut-off", linewidth=4)

plt.legend(title = 'Samples')
plt.xlabel('Test score')
plt.ylabel('Density')

In [ ]:
ttest_ind(losses, p_losses)

In [ ]:
def plot_cm(labels, predictions, p=0.5):
  cm = confusion_matrix(labels, predictions > p)
  plt.figure(figsize=(8,8))
  sns.heatmap(cm, annot=True, fmt="d")
  plt.ylabel('Actual label')
  plt.xlabel('Predicted label')

  print('Absent Disease Detected (True Negatives): ', cm[0][0])
  print('Absent Disease Incorrectly Detected (False Positives): ', cm[0][1])
  print('Case Missed (False Negatives): ', cm[1][0])
  print('Case Detected (True Positives): ', cm[1][1])
  print('Total Cases : ', np.sum(cm[1]))

In [ ]:
all_losses = np.concatenate((losses, p_losses), axis=None)
all_labels = np.concatenate(([0] * len(losses), [1] * len(p_losses)), axis=None)

plot_cm(all_labels, all_losses, cutoff)

# Voting system

In [ ]:
test_pos_latent = full_encoder.predict(test_pos_df, batch_size=50)
test_neg_latent = full_encoder.predict(test_neg_df, batch_size=50)

In [ ]:
best_frac = 0.75
cutoff = 0.996531000000044

voters, testers = train_test_split(test_pos_latent, test_size=best_frac, random_state=RNG_SEED)

pos_votes = cosine_similarity(testers, voters)
neg_votes = cosine_similarity(test_neg_latent, voters)
pos_mean = np.median(pos_votes, axis=1)
neg_mean = np.median(neg_votes, axis=1) 

sns.histplot(data=neg_mean, label="Negative cases", color='darkorange', kde=True, stat='density', cbar_kws={'edgecolor':'black'}, line_kws={'linewidth': 4})
sns.histplot(data=pos_mean, label="Positive cases", color='darkblue', kde=True, stat='density', cbar_kws={'edgecolor':'black'}, line_kws={'linewidth': 4}, bins=25)

plt.legend(title = 'Samples')
plt.xlabel('Test score')
plt.ylabel('Density')

In [ ]:
ttest_ind(neg_mean, pos_mean)

In [ ]:
def generate_metrics(cutoff, pos_mean, neg_mean):
  fp = (neg_mean > cutoff).sum()
  tn = len(neg_mean) - fp

  tp = (pos_mean > cutoff).sum()
  fn = len(pos_mean) - tp

  prevalence = (tp + fn) / (tp + fn + tn + fp)
  sensitivity = tp / (tp + fn)
  fnr = fn / (tp + fn)
  tnr = tn / (fp + tn)
  fallout = fp / (fp + tn)
  ppv = tp / (tp + fp)
  npv = tn / (fn + tn)

  lr_p = sensitivity / fallout
  lr_n = fnr / tnr
  dor = lr_p / lr_n
  f_score = 2 * (ppv * sensitivity) / (ppv + sensitivity)

  return prevalence, tnr, sensitivity, fallout, ppv, npv, lr_p, lr_n, dor, f_score

In [ ]:
cutoff = 0.996531

prevalence, tnr, sensitivity, fallout, ppv, npv, lr_p, lr_n, dor, f_score = generate_metrics(cutoff,pos_mean,neg_mean)

print("cutoff: ", cutoff)
print("Prevalence ", prevalence)
print("Specificity ", tnr)
print("True positive rate ", sensitivity)
print("False positive rate ", fallout)
print("PPV ", ppv)
print("NPV ", npv)
print("LR+ ", lr_p)
print("LR- ", lr_n)
print("DOR ", dor)
print("F-score ", f_score)

In [ ]:
all_losses = np.concatenate((neg_mean, pos_mean), axis=None)
all_labels = np.concatenate(([0] * len(neg_mean), [1] * len(pos_mean)), axis=None)

plot_cm(all_labels, all_losses, cutoff)

# Combined

In [ ]:
X, Y = train_test_split(p_losses, test_size=best_frac, random_state=RNG_SEED)

neg_comb = neg_mean + losses
pos_comb = pos_mean + Y

In [ ]:
def generate_metrics_combined(neg_mean, pos_mean, neg_loss, pos_loss):
  fp = ((neg_mean > 0.996531000000044) | (neg_loss > 0.052)).sum()
  tn = len(neg_mean) - fp

  tp = ((pos_mean > 0.996531000000044) | (pos_loss > 0.052)).sum()
  fn = len(pos_mean) - tp

  prevalence = (tp + fn) / (tp + fn + tn + fp)
  sensitivity = tp / (tp + fn)
  fnr = fn / (tp + fn)
  tnr = tn / (fp + tn)
  fallout = fp / (fp + tn)
  ppv = tp / (tp + fp)
  npv = tn / (fn + tn)

  lr_p = sensitivity / fallout
  lr_n = fnr / tnr
  dor = lr_p / lr_n
  f_score = 2 * (ppv * sensitivity) / (ppv + sensitivity)

  return prevalence, tnr, sensitivity, fallout, ppv, npv, lr_p, lr_n, dor, f_score

In [ ]:
prevalence, tnr, sensitivity, fallout, ppv, npv, lr_p, lr_n, dor, f_score = generate_metrics_combined(neg_mean, pos_mean, losses, Y)

print("Prevalence ", prevalence)
print("Specificity ", tnr)
print("True positive rate ", sensitivity)
print("False positive rate ", fallout)
print("PPV ", ppv)
print("NPV ", npv)
print("LR+ ", lr_p)
print("LR- ", lr_n)
print("DOR ", dor)
print("F-score ", f_score)

In [ ]:
all_losses = np.concatenate((((neg_mean > 0.996531000000044) | (losses > 0.052)), ((pos_mean > 0.996531000000044) | (Y > 0.052))), axis=None)
all_labels = np.concatenate(([0] * len(neg_mean), [1] * len(pos_mean)), axis=None)

cm = confusion_matrix(all_labels, all_losses)
plt.figure(figsize=(8,8))
sns.heatmap(cm, annot=True, fmt="d")
plt.ylabel('Actual label')
plt.xlabel('Predicted label')